In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
%pip install mlflow
%pip install dagshub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:

import mlflow
import mlflow.sklearn
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, classification_report,
    precision_recall_curve, roc_curve, average_precision_score
)
import dagshub
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer

In [4]:
# Initialize MLflow tracking
try:
    # Initialize Dagshub
    dagshub.init(repo_owner='konstantine25b', repo_name='IEEE-CIS-Fraud-Detection', mlflow=True)
    print("DagsHub initialized successfully.")
    mlflow.set_experiment("IEEE-CIS-Fraud-Detection_decision_tree")
    print(f"MLflow experiment set to: {mlflow.get_experiment_by_name('IEEE-CIS-Fraud-Detection_decision_tree').name}")
    mlflow_active = True
except Exception as e:
    print(f"Could not initialize DagsHub or set MLflow experiment: {e}")
    print("Proceeding without MLflow tracking.")
    mlflow_active = False

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d8208cf0-85fa-4f02-894f-facdc90fa8ac&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=e9486f7f4e294101abeff5217123fda13ed9e617f4fabfff487702043ec6e848




Accessing as konstantine25b

Initialized MLflow to track repo "konstantine25b/IEEE-CIS-Fraud-Detection"

Repository konstantine25b/IEEE-CIS-Fraud-Detection initialized!

2025/04/20 11:51:07 INFO mlflow.tracking.fluent: Experiment with name 'IEEE-CIS-Fraud-Detection_decision_tree' does not exist. Creating a new experiment.


DagsHub initialized successfully.
MLflow experiment set to: IEEE-CIS-Fraud-Detection_decision_tree


In [5]:
# Start MLflow run
run_name = f"decision_tree_{time.strftime('%Y%m%d_%H%M%S')}"
if mlflow_active:
    mlflow.start_run(run_name=run_name)
    print(f"MLflow run started with name: {run_name}")

MLflow run started with name: decision_tree_20250420_115131


# Loading Original Data

In [6]:
print("\n--- Loading Original Data from Kaggle ---")
try:
    identity_df = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
    transaction_df = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
    print(f"Loaded identity data shape: {identity_df.shape}")
    print(f"Loaded transaction data shape: {transaction_df.shape}")
except FileNotFoundError:
    print("Error: One or both of the CSV files were not found. Please make sure the file paths are correct.")
    if mlflow_active:
        mlflow.end_run()
    exit()


--- Loading Original Data from Kaggle ---
Loaded identity data shape: (144233, 41)
Loaded transaction data shape: (590540, 394)


# Loading Preprocessing Pipeline from MLflow

In [7]:
preprocessing_run_id = '962cdbe1451f4abe864ff349e123e7de'  # Example run ID
try:
    # Load the transaction pipeline model
    transaction_pipeline = mlflow.sklearn.load_model(f'runs:/{preprocessing_run_id}/transaction_pipeline_model')
    print("Loaded transaction preprocessing pipeline from MLflow.")
    
    # Load the identity pipeline model if it exists
    try:
        identity_pipeline = mlflow.sklearn.load_model(f'runs:/{preprocessing_run_id}/identity_pipeline_model')
        print("Loaded identity preprocessing pipeline from MLflow.")
        identity_pipeline_exists = True
    except:
        print("Identity preprocessing pipeline not found. Will only use transaction pipeline.")
        identity_pipeline_exists = False
    
    # Load the feature selection information
    try:
        selected_features = mlflow.artifacts.load_text(f'runs:/{preprocessing_run_id}/selected_features.txt').strip().split('\n')
        print(f"Loaded {len(selected_features)} selected features from MLflow.")
    except:
        print("Selected features list not found. Will use all features after preprocessing.")
        selected_features = None
except Exception as e:
    print(f"Error loading preprocessing pipeline from MLflow: {e}")
    print("Please ensure you have the correct run ID and the pipeline is properly saved.")
    if mlflow_active:
        mlflow.end_run()
    exit()


Loaded transaction preprocessing pipeline from MLflow.


Loaded identity preprocessing pipeline from MLflow.


Loaded 50 selected features from MLflow.


In [8]:
# Split the data
X_transaction = transaction_df.drop('isFraud', axis=1)
y_transaction = transaction_df['isFraud']

X_train, X_test, y_train, y_test = train_test_split(
    X_transaction, y_transaction, test_size=0.2, random_state=42, stratify=y_transaction
)
print(f"Train set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

Train set shape: (472432, 393)
Test set shape: (118108, 393)


In [9]:
# Get the original feature names before preprocessing
original_feature_names = X_train.columns.tolist()
print(f"Original feature count: {len(original_feature_names)}")

# Apply the transaction preprocessing pipeline
print("Applying transaction preprocessing pipeline...")
X_train_processed = transaction_pipeline.transform(X_train)
X_test_processed = transaction_pipeline.transform(X_test)

# Convert to DataFrame with feature names that match the preprocessing pipeline
# First, use generic names
feature_names = [f"feature_{i}" for i in range(X_train_processed.shape[1])]
X_train_final = pd.DataFrame(X_train_processed, columns=feature_names)
X_test_final = pd.DataFrame(X_test_processed, columns=feature_names)


Original feature count: 393
Applying transaction preprocessing pipeline...


# Apply feature selection if available

In [10]:
if selected_features is not None:
    print(f"Applying feature selection to keep {len(selected_features)} features...")
    
    # Print some of the selected feature names to understand their format
    print(f"Sample selected feature names: {selected_features[:5]}")
    
    # Check if any of the selected features match the original feature names
    original_matches = [f for f in selected_features if f in original_feature_names]
    if original_matches:
        print(f"Found {len(original_matches)} features that match original feature names.")
        
        # Create a mapping from original feature names to processed feature indices
        # This is a simplified approach - in reality, the mapping might be more complex
        feature_mapping = {}
        for i, feature in enumerate(original_feature_names):
            if i < len(feature_names):
                feature_mapping[feature] = feature_names[i]
        
        # Map selected features to their corresponding processed features
        mapped_features = []
        for feature in selected_features:
            if feature in feature_mapping:
                mapped_features.append(feature_mapping[feature])
        
        if mapped_features:
            print(f"Mapped {len(mapped_features)} selected features to processed features.")
            
            # Apply the selection
            X_train_final = X_train_final[mapped_features]
            X_test_final = X_test_final[mapped_features]
        else:
            print("Could not map any selected features to processed features.")
            print("Using all processed features.")
    else:
        # If selected features don't match original names, they might be indices or have a different format
        print("Selected features don't match original feature names.")
        
        # Print all selected features to understand their format
        print(f"First 5 selected features: {selected_features[:5]}")
        print(f"Last 5 selected features: {selected_features[-5:]}")
        
        # Check what columns are actually available in the processed data
        print(f"Available columns in processed data: {X_train_final.columns[:10]}...")
        
        # Extract the indices from the selected feature names
        feature_indices = []
        for feature in selected_features:
            # Try different patterns to extract indices
            if '_x' in feature or '_y' in feature:
                # Format like '0_x' or '1_y'
                parts = feature.split('_')
                if parts[0].isdigit():
                    feature_indices.append(int(parts[0]))
            elif feature.isdigit():
                # Format like '68'
                feature_indices.append(int(feature))
        
        # Get the available columns in the processed data
        available_columns = X_train_final.columns.tolist()
        
        # Create a new DataFrame with the original feature names
        X_train_renamed = pd.DataFrame()
        X_test_renamed = pd.DataFrame()
        
        # Map each selected feature to a column in the processed data
        for i, feature in enumerate(selected_features):
            if i < len(available_columns):
                # Use the i-th column from the processed data for the i-th selected feature
                # Extract as a Series using .iloc to avoid the DataFrame issue
                X_train_renamed[feature] = X_train_final.iloc[:, i].values
                X_test_renamed[feature] = X_test_final.iloc[:, i].values
            else:
                print(f"Warning: Not enough columns in processed data for feature {feature}")
        
        # Use the renamed DataFrames
        X_train_final = X_train_renamed
        X_test_final = X_test_renamed
        
        print(f"Final train set shape after preprocessing: {X_train_final.shape}")
        print(f"Final test set shape after preprocessing: {X_test_final.shape}")
        print(f"Feature names in final dataset: {X_train_final.columns.tolist()[:5]}...")
else:
    print("No selected features provided. Using all features from the preprocessing pipeline.")

print(f"Final train set shape after preprocessing: {X_train_final.shape}")
print(f"Final test set shape after preprocessing: {X_test_final.shape}")


Applying feature selection to keep 50 features...
Sample selected feature names: ['0_x', '1_x', '2_x', '3_x', '4_x']
Selected features don't match original feature names.
First 5 selected features: ['0_x', '1_x', '2_x', '3_x', '4_x']
Last 5 selected features: ['6_y', '7_y', '8_y', '30_y', '31_y']
Available columns in processed data: Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9'],
      dtype='object')...
Final train set shape after preprocessing: (472432, 50)
Final test set shape after preprocessing: (118108, 50)
Feature names in final dataset: ['0_x', '1_x', '2_x', '3_x', '4_x']...
Final train set shape after preprocessing: (472432, 50)
Final test set shape after preprocessing: (118108, 50)


In [11]:
print("\n--- Checking for NaN Values in Processed Data ---")
train_nan_count = X_train_final.isna().sum().sum()
test_nan_count = X_test_final.isna().sum().sum()

print(f"Number of NaN values in training data: {train_nan_count}")
print(f"Number of NaN values in testing data: {test_nan_count}")



--- Checking for NaN Values in Processed Data ---
Number of NaN values in training data: 0
Number of NaN values in testing data: 0


In [12]:
if train_nan_count > 0 or test_nan_count > 0:
    print("WARNING: NaN values found in processed data!")
    
    # Get columns with NaN values
    train_nan_cols = X_train_final.columns[X_train_final.isna().any()].tolist()
    test_nan_cols = X_test_final.columns[X_test_final.isna().any()].tolist()
    
    if train_nan_cols:
        print(f"Training data columns with NaN values: {train_nan_cols}")
        print(f"NaN counts per column: \n{X_train_final[train_nan_cols].isna().sum()}")
    
    if test_nan_cols:
        print(f"Testing data columns with NaN values: {test_nan_cols}")
        print(f"NaN counts per column: \n{X_test_final[test_nan_cols].isna().sum()}")
    
    # Fill NaN values with median as a quick fix
    print("Filling NaN values with column medians...")
    for col in train_nan_cols:
        median_val = X_train_final[col].median()
        X_train_final[col] = X_train_final[col].fillna(median_val)
    
    for col in test_nan_cols:
        if col in X_train_final.columns:
            median_val = X_train_final[col].median()
        else:
            median_val = X_test_final[col].median()
        X_test_final[col] = X_test_final[col].fillna(median_val)
    
    # Verify NaN values are gone
    train_nan_count_after = X_train_final.isna().sum().sum()
    test_nan_count_after = X_test_final.isna().sum().sum()
    print(f"NaN values after filling - training data: {train_nan_count_after}")
    print(f"NaN values after filling - testing data: {test_nan_count_after}")
else:
    print("No NaN values found in processed data. Preprocessing pipeline handled missing values correctly.")


No NaN values found in processed data. Preprocessing pipeline handled missing values correctly.


In [14]:
if mlflow_active:
    mlflow.log_param("original_features", X_train.shape[1])
    mlflow.log_param("final_features", X_train_final.shape[1])


# Cross-Validation

In [15]:
print("\n--- Setting up Cross-Validation and Hyperparameter Tuning ---")

# Create a pipeline with scaling and decision tree
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Define a minimal hyperparameter grid for quick training
param_grid = {
    'classifier__max_depth': [5, 10],  # Just two depth options
    'classifier__min_samples_split': [10],  # Only one value
    'classifier__class_weight': ['balanced']  # Only one class weight option
}


--- Setting up Cross-Validation and Hyperparameter Tuning ---


In [16]:
# Reduce cross-validation folds
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Add this code before grid search to sample the training data
# Sample 10% of the training data for hyperparameter tuning
sample_size = int(0.1 * len(X_train_final))
indices = np.random.choice(len(X_train_final), sample_size, replace=False)
X_train_sample = X_train_final.iloc[indices]
y_train_sample = y_train.iloc[indices]


In [17]:
print(f"Using {len(X_train_sample)} samples for hyperparameter tuning...")
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=cv, 
    scoring='average_precision',  # Optimize for average precision
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X_train_sample, y_train_sample)


Using 47243 samples for hyperparameter tuning...
Fitting 3 folds for each of 2 candidates, totalling 6 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        DecisionTreeClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__class_weight': ['balanced'],
                         'classifier__max_depth': [5, 10],
                         'classifier__min_samples_split': [10]},
             scoring='average_precision', verbose=1)

In [18]:
# After finding best parameters, train final model on full dataset
print("Training final model with best parameters on full dataset...")
best_params = grid_search.best_params_
final_model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeClassifier(
        max_depth=best_params['classifier__max_depth'],
        min_samples_split=best_params['classifier__min_samples_split'],
        class_weight=best_params['classifier__class_weight'],
        random_state=42
    ))
])
final_model.fit(X_train_final, y_train)
model = final_model  # Use this for predictions


Training final model with best parameters on full dataset...


In [19]:
if mlflow_active:
    mlflow.log_params(best_params)
    mlflow.log_metric("best_cv_score", grid_search.best_score_)
    
    # Log all CV results
    cv_results = pd.DataFrame(grid_search.cv_results_)
    mlflow.log_text(cv_results.to_string(), "cv_results.txt")




In [20]:
# Evaluate the model
print("\n--- Evaluating Model Performance ---")
y_pred = model.predict(X_test_final)
y_pred_proba = model.predict_proba(X_test_final)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
avg_precision = average_precision_score(y_test, y_pred_proba)

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Average Precision: {avg_precision:.4f}")



--- Evaluating Model Performance ---
Accuracy: 0.8381
Precision: 0.1486
Recall: 0.7672
F1 Score: 0.2490
ROC AUC: 0.8744
Average Precision: 0.4448


In [21]:
# Log metrics
if mlflow_active:
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("avg_precision", avg_precision)

# Print classification report
print("\nClassification Report:")
report = classification_report(y_test, y_pred)
print(report)

# Log classification report
if mlflow_active:
    mlflow.log_text(report, "classification_report.txt")



Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.84      0.91    113975
           1       0.15      0.77      0.25      4133

    accuracy                           0.84    118108
   macro avg       0.57      0.80      0.58    118108
weighted avg       0.96      0.84      0.89    118108



In [22]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
cm_path = "confusion_matrix.png"
plt.savefig(cm_path)
plt.close()

# Create and log ROC curve
plt.figure(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.tight_layout()
roc_path = "roc_curve.png"
plt.savefig(roc_path)
plt.close()

# Create and log Precision-Recall curve
plt.figure(figsize=(8, 6))
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
plt.plot(recall_curve, precision_curve, label=f'PR Curve (AP = {avg_precision:.4f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.tight_layout()
pr_path = "precision_recall_curve.png"
plt.savefig(pr_path)
plt.close()


In [23]:
# Log artifacts
if mlflow_active:
    mlflow.log_artifact(cm_path)
    mlflow.log_artifact(roc_path)
    mlflow.log_artifact(pr_path)
    
    # Log the model
    mlflow.sklearn.log_model(model, "decision_tree_model")
    
    # Log feature importances
    feature_importances = pd.DataFrame({
        'Feature': X_train_final.columns,
        'Importance': model.named_steps['classifier'].feature_importances_
    }).sort_values('Importance', ascending=False)
    
    mlflow.log_text(feature_importances.to_string(), "feature_importances.txt")
    
    # Create and log feature importance plot
    plt.figure(figsize=(12, 10))
    top_n = 20
    top_importances = feature_importances.head(top_n)
    plt.barh(top_importances['Feature'], top_importances['Importance'])
    plt.title(f'Top {top_n} Feature Importances')
    plt.xlabel('Importance')
    plt.tight_layout()
    importance_path = "feature_importances.png"
    plt.savefig(importance_path)
    plt.close()
    
    mlflow.log_artifact(importance_path)

# Log NaN check results
if mlflow_active:
    mlflow.log_param("train_nan_count_before", train_nan_count)
    mlflow.log_param("test_nan_count_before", test_nan_count)
    if train_nan_count > 0 or test_nan_count > 0:
        mlflow.log_param("train_nan_count_after", train_nan_count_after)
        mlflow.log_param("test_nan_count_after", test_nan_count_after)

# End MLflow run
if mlflow_active:
    mlflow.end_run()
    print("MLflow run completed and artifacts logged.")


2025/04/20 11:57:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run decision_tree_20250420_115131 at: https://dagshub.com/konstantine25b/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/3/runs/f54a6bf3c1744e4ea742ff8fc3e647d9
🧪 View experiment at: https://dagshub.com/konstantine25b/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/3
MLflow run completed and artifacts logged.
